This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [1]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction, plot_utils, fisher

flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

In [3]:
data_velocity = pd.read_parquet(data_path / "data_velocity.parquet")

coordinates_velocity = np.array([data_velocity["ra"], data_velocity["dec"], data_velocity["rcom_zobs"]])

data_velocity_true = {**data_velocity.to_dict(orient='list'), 
                      "velocity_error": np.zeros(len(data_velocity["velocity"].values))}


# Create the data Vector
DataTrueVel = data_vector.DirectVel(data_velocity_true)

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


In [4]:
DataVel = data_vector.VelFromIntrinsicScatter(data_velocity.to_dict(orient='list'))
print("Data free parameters: ", DataVel.free_par)


Data free parameters:  ['sigma_M']


In [5]:
ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")
sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}

### Compute covariance
size_batch = 10_000
number_worker = 16

covariance_fisher = covariance.CovMatrix.init_from_flip(
    "carreres23",
    "velocity",
    power_spectrum_dict,
    coordinates_velocity=coordinates_velocity,
    size_batch=size_batch,
    number_worker=number_worker,
)

print("Covariance free parameters: ", covariance_fisher.free_par)


/global/common/software/nersc/pe/conda-envs/24.1.0/python-3.11/nersc-python/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
[ 000007.32 ]: 01-23 03:31  root            INFO     Covariance matrix generated from flip with carreres23 model in 5.93e-01 seconds


Covariance free parameters:  ['sigv', 'fs8']


In [6]:
fisher_properties = {
    "inversion_method": "inverse",
}


In [7]:
parameter_dict = {
    "fs8": 0.55,
    "sigv": 200,
    "sigma_M": 0.12
}


Fisher = fisher.FisherMatrix.init_from_covariance(
    covariance_fisher,
    DataVel,
    parameter_dict,
    fisher_properties=fisher_properties,
)

parameter_name_list, fisher_matrix = Fisher.compute_fisher_matrix()

error = np.sqrt(np.diag(np.linalg.pinv(fisher_matrix)))

In [8]:
for p, e in zip(parameter_name_list, error):
    print(f"Fisher error on {p} = {e:.4f}")

Fisher error on fs8 = 0.0960
